In [ ]:
# 수정 전

#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
import time

# 크롤링할 url 생성하는 함수 만들기(검색어, 시작 날짜, 종료 날짜, 최대 페이지)
def makeUrl(search, s_date, e_date, maxpage):
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1  
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지
    urls = []
    for i in range(maxpage_t):
        url = "https://search.naver.com/search.naver?where=news&query=" + search + "&sort=0&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        urls.append(url)
        page += 10
    return urls    

# html에서 원하는 속성 추출하는 함수 만들기(기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html 생성해서 기사 크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스 크롤링 시작#####
search = input("검색어 입력: ")  
s_date = input("시작 날짜 입력(예시: 2019.01.04): ")  #2019.01.04
e_date = input("종료 날짜 입력(예시: 2019.01.05): ")   #2019.01.05
maxpage = input("최대 크롤링할 페이지 수를 입력하세요: ")

# naver url 생성
url = makeUrl(search, s_date, e_date, maxpage)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
news_names = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass


# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    time.sleep(2)

    # 언론사명 가져 오기
    names = news_html.select('#contents > div.copyright > div > p')
    if names:
      name = names[0].string[12:-38]
    elif names != names:
      time.sleep(2)
      names = news_html.select('#content > div.end_ct > div > div.copyright > div > p')
      name = names[0].string[12:-38]
    else:
      name = ''

    # 뉴스 제목 가져 오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져 오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져 오기:  list합치기
    content = ''.join(str(content))

    # html 태그 제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_names.append(name)
    news_titles.append(title)
    news_contents.append(content)

# 날짜 가져 오기
    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time'][:10]
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='', string=str(news_date))

    news_dates.append(news_date)


###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates, 'name': news_names, 'title':news_titles, 'content':news_contents, 'link': final_urls})

#중복 행 지우기
# news_df = news_df.drop_duplicates(keep='first',ignore_index=True)

#데이터 프레임을 엑셀 파일로 저장
outputFileName = f'{s_date} ~ {e_date}.xlsx'
news_df.to_excel(outputFileName, sheet_name='sheet1')

In [ ]:
# 간략화

#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
import time

# 크롤링할 url 생성하는 함수 만들기(검색어, 시작 날짜, 종료 날짜, 최대 페이지)
def makeUrl(search, s_date, e_date, maxpage):
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    url = [f"https://search.naver.com/search.naver?where=news&query={search}&sort=0&ds={s_date}&de={e_date}&nso=so%3Ar%2Cp%3Afrom{s_from}to{e_to}%2Ca%3A&start={i}" for i in range(1, int(maxpage) * 10, 10)]
    return url  

# html에서 원하는 속성 추출하는 함수 만들기(기사, 추출하려는 속성값)
def news_attrs_crawler(articles, attrs):
    attrs_content = [i.attrs[attrs] for i in articles]
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html 생성해서 기사 크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스 크롤링 시작#####
search = input("검색어 입력: ")  
s_date = input("시작 날짜 입력(예시: 2019.01.04): ")  #2019.01.04
e_date = input("종료 날짜 입력(예시: 2019.01.05): ")   #2019.01.05
maxpage = input("최대 크롤링할 페이지 수를 입력하세요: ")

# naver url 생성
url = makeUrl(search, s_date, e_date, maxpage)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
news_names = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass


# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    time.sleep(2)

    # 언론사명 가져 오기
    names = news_html.select('#contents > div.copyright > div > p')
    if names:
      name = names[0].string[12:-38]
    elif names != names:
      time.sleep(2)
      names = news_html.select('#content > div.end_ct > div > div.copyright > div > p')
      name = names[0].string[12:-38]
    else:
      name = ''

    # 뉴스 제목 가져 오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져 오기
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져 오기:  list합치기
    content = ''.join(str(content))

    # html 태그 제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>' # 태그 제거
    pattern2 = r'[\n\t<>]|&lt;|&gt;' # 특수 기호 제거

    title = re.sub(pattern=pattern1, repl='', string=str(title))
    title = re.sub(pattern=pattern2, repl='', string=str(title))

    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern3 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern3, '')
    content = re.sub(pattern2, '', content)[1:-1] # 기사 본문 맨 앞, 맨 뒤 [ ] 제거

    news_names.append(name)
    news_titles.append(title)
    news_contents.append(content)

# 날짜 가져 오기
    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time'][:10]
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='', string=str(news_date))

    # news_dates.append(news_date)
    news_dates.append(news_date[:10].replace('.', '-'))


###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates, 'name': news_names, 'title':news_titles, 'content':news_contents, 'link': final_urls})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)

#데이터 프레임을 엑셀 파일로 저장
outputFileName = f'{s_date} ~ {e_date}.xlsx'
news_df.to_excel(outputFileName, sheet_name='sheet1')

검색어 입력: 서울 아파트 매매
시작 날짜 입력(예시: 2019.01.04): 2020.04.01
종료 날짜 입력(예시: 2019.01.05): 2020.06.30
최대 크롤링할 페이지 수를 입력하세요: 300


100%|██████████| 1550/1550 [1:07:08<00:00,  2.60s/it]


In [ ]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1335 entries, 0 to 1334
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     1335 non-null   object
 1   name     1335 non-null   object
 2   title    1335 non-null   object
 3   content  1335 non-null   object
 4   link     1335 non-null   object
dtypes: object(5)
memory usage: 52.3+ KB


In [ ]:
news_df

,date,name,title,content,link
0,2020-03-31,연합뉴스,대출 중단 등 악재에도 2월 서울 아파트 거래량 '깜짝' 증가,"강남 급매물 거래, 강북 풍선효과로 1월 대비 25% 늘어15억 초과 거래비중도 증...",https://n.news.naver.com/mnews/article/001/001...
1,2020-03-31,뉴스1,강남3구 아파트 값 하락세,(서울=뉴스1) 이승배 기자 = 강력한 대출 규제를 낀 정부의 12·16 부동산대책...,https://n.news.naver.com/mnews/article/421/000...
2,2020-03-29,조선일보,쌀값 3배 오른 동안 84배 상승한 강남 아파트,하나금융경영연구소 보고서최저임금도 GDP보다 상승폭이 더 커40년간 1인당 GDP와...,https://n.news.naver.com/mnews/article/023/000...
3,2020-03-31,국민일보,수억 값싼 급매물 출현… 서울 아파트 매매가격 하락세 돌입,지난해 6월 첫주 이후 처음… 코로나19 사태·규제 강화로 시장의 불안감 커졌기 때...,https://n.news.naver.com/mnews/article/005/000...
4,2020-03-31,아시아경제,코로나에도 늘어난 서울 아파트 거래,"지난달 서울 아파트 매매 계약 건수 8074건전월 대비 24.7%, 전년 동월 대비...",https://n.news.naver.com/mnews/article/277/000...
...,...,...,...,...,...
1126,2020-01-29,더팩트,"자이·힐스테이트·래미안…아파트 브랜드, 집값 얼마나 좌우할까",브랜드 아파트에 대한 수요층의 관심이 갈수록 두드지러지고 있다. /더팩트 DB브랜드...,https://n.news.naver.com/mnews/article/629/000...
1127,2020-01-23,헤럴드경제,12·16 규제 역풍맞은 이낙연 강남 아파트,이달 초에 전세 계약 체결최초 매물 등록은 지난달 11일최근 호가 낮췄지만 거래 끊...,https://n.news.naver.com/mnews/article/016/000...
1128,2020-01-28,디지털타임스,"지방 큰손, 상경투자 급증세… 9억 이하 아파트 집중 매입",작년 3687건 달해… 26% 차지중랑·구로 등 외곽지역서 활발서울의 한 부동산중개...,https://n.news.naver.com/mnews/article/029/000...
1129,2020-01-29,한국경제,집값 오르자…미분양 아파트도 웃돈,2~3년 전엔 눈물의 분양이젠 수천만원 프리미엄경기 용인 남사면에 들어선 'e편한세...,https://n.news.naver.com/mnews/article/015/000...


In [ ]:
import re

string = "\n\t\t\t'정산회담' 김나희,"

pattern = r'[\n\t<>]'
string = re.sub(pattern, '', string)[1:-1]

In [ ]:
pattern = r'[\n\t<>\\]|&lt;|&gt;'
news_df.loc[10, 'content'] = re.sub(pattern, '', news_df.loc[10, 'content'])[1:-1]

In [ ]:
news_df.loc[10, 'content']

'지역별 매수우위지수 주간추이 / 사진=KB부동산 리브온오는 6월말까지 한시적 양도세 중과세 배제정책으로 몇몇 다주택자 물건이 초급매로 나오는 모양새지만, 서울 아파트 매수문의가 3월 셋째주 기준점인 100보다 낮은 91.8을 기록한 후 넷째주에는 더 하락해 81.1을 기록했습니다. 특히 강남 지역은 73.1로 매수 문의가 대폭 줄었습니다.전국 기준 매수문의지수도 지난주 74.3에서 이번주 66.9를 기록해 시장이 정부의 부동산 정책과 코로나19 장기화 영향을 받아 경색되고 있다는 진단이 나옵니다.오늘(30일) KB주택시장동향자료에 따르면, 3월 넷째주 전국 매수우위지수는 66.9를 기록해 전주(74.3)대비 크게 하락했습니다.코로나 사태로 인해 시장에서 매수에 대한 문의도 위축된 것으로 보입니다. 이 지수는 100을 초과하면 \'매수자가 많다\'를, 100 미만일 경우 \'매도자가 많다\'를 의미합니다. 서울도 81.1로 전주(91.8)대비 크게 하락했습니다. 강남지역은 지난주 82.8에서 73.1로 하락했고, 강북지역도 지난주(102.0)대비 하락한 90.2를 기록하며 매수문의가 대폭 줄어들었습니다.아파트 매매가격 지구는 인천(0.28%)과 경기(0.20%), 서울(0.06%)은 상승폭이 지난주보다 줄었고, 인천을 제외한 5개 광역시(0.07%)는 대전(0.29%)만 다소 높은 편입니다. 울산(0.06%), 부산(0.05%), 대구(0.03%)는 소폭 상승했고 광주(0.00%)는 보합을 보였습니다.수도권(0.16%)은 전주대비 상승했고, 광역시 이외의 기타 지방(-0.01%)은 하락했습니다. 세종(0.16%)만 상승했으며 경북(-0.04%), 경남(-0.02%), 충북(-0.02%), 강원(-0.01%), 전북(-0.01%)은 소폭 하락한 반면 충남과 전남은 보합(0.00%)을 기록했습니다.서울 아파트 매매가격은 전주 대비 0.06%로 지난주의 상승 0.12%보다 상승세가 크게 둔화되면서 보합권에 진입했습니다. 특히 강남구(-0.14%), 서초구(-0.04

In [ ]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1388 entries, 0 to 1387
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     1388 non-null   object
 1   name     1388 non-null   object
 2   title    1388 non-null   object
 3   content  1388 non-null   object
 4   link     1388 non-null   object
dtypes: object(5)
memory usage: 54.3+ KB


In [ ]:
news_df.head()

,date,name,title,content,link
0,2020-03-31,연합뉴스,대출 중단 등 악재에도 2월 서울 아파트 거래량 '깜짝' 증가,"[\n\t\t\t강남 급매물 거래, 강북 풍선효과로 1월 대비 25% 늘어15억 초...",https://n.news.naver.com/mnews/article/001/001...
1,2020-03-31,뉴스1,강남3구 아파트 값 하락세,[\n\n\n\n\n(서울=뉴스1) 이승배 기자 = 강력한 대출 규제를 낀 정부의 ...,https://n.news.naver.com/mnews/article/421/000...
2,2020-03-29,조선일보,쌀값 3배 오른 동안 84배 상승한 강남 아파트,[\n하나금융경영연구소 보고서최저임금도 GDP보다 상승폭이 더 커\n\n\n\n40...,https://n.news.naver.com/mnews/article/023/000...
3,2020-03-31,국민일보,수억 값싼 급매물 출현… 서울 아파트 매매가격 하락세 돌입,[\n지난해 6월 첫주 이후 처음… 코로나19 사태·규제 강화로 시장의 불안감 커졌...,https://n.news.naver.com/mnews/article/005/000...
4,2020-03-31,아시아경제,코로나에도 늘어난 서울 아파트 거래,"[\n지난달 서울 아파트 매매 계약 건수 8074건전월 대비 24.7%, 전년 동월...",https://n.news.naver.com/mnews/article/277/000...


In [ ]:
news_df[news_df.duplicated(keep = False)].sort_values(by = 'title')

,date,name,title,content,link


In [ ]:
news_df[news_df.duplicated(keep = False)]['title'].value_counts()

9510가구에 매매 연 5건…1채 팔리자 시세 2억 뛰었다    6
서울 아파트 가격 상승세 주춤… 수원 영통·안양 만안 급등    5
서울 집값, 조정 문턱…상반기에 살까, 하반기까지 버틸까     4
서울 아파트 급매물에도 ‘거래실종’ 심화              4
지난달(01월) 아파트 매매 실거래가                4
                                   ..
서울 아파트 상위 10% 평균 매매가 사상 첫 20억 돌파    2
'동학개미'가 부동산으로 발길을 돌린다면 [여기는 논설실]    2
서울 아파트 매매 2건 중 1건 꼴로 ‘편법 증여’        2
서울 아파트 전세 상승률, 매매 상승세 앞질렀다          2
집값 오르자…미분양 아파트도 웃돈                  2
Name: title, Length: 207, dtype: int64

In [ ]:
news_df[news_df['name'] == '']

,date,name,title,content,link
1253,2020-05-25,,"‘구해줘 홈즈' 이현이X임성빈X노홍철, 매매가 4억 원대 ‘초코우유 하우스’ 승리",[OSEN=강서정 기자] '구해줘! 홈즈'에서는 덕팀의 ‘초.코.우.유 하우스’가...,https://n.news.naver.com/mnews/article/109/000...


In [ ]:
news_df.loc[304, 'date'][:10].replace('.', '-')

'2020-03-11'

In [ ]:
news_df[news_df['name'] == ''].count()

date       10
name       10
title      10
content    10
link       10
dtype: int64

In [ ]:
news_df[news_df['name'] == '']['link'].values

array(['https://n.news.naver.com/mnews/article/421/0004528266?sid=106',
       'https://n.news.naver.com/mnews/article/311/0001124417?sid=106',
       'https://n.news.naver.com/mnews/article/311/0001125532?sid=106',
       'https://n.news.naver.com/mnews/article/003/0009740646?sid=102',
       'https://n.news.naver.com/mnews/article/609/0000258718?sid=106',
       'https://n.news.naver.com/mnews/article/609/0000259501?sid=106',
       'https://n.news.naver.com/mnews/article/277/0004645150?sid=106',
       'https://n.news.naver.com/mnews/article/015/0004304426?sid=106',
       'https://n.news.naver.com/mnews/article/076/0003540534?sid=106',
       'https://n.news.naver.com/mnews/article/609/0000259765?sid=106'],
      dtype=object)

In [ ]:
news_df[news_df['content'] == '']

,date,name,title,content,link


In [ ]:
news_df[news_df['title'] == '']

,date,name,title,content,link


In [ ]:
news_df[news_df['title'].isna()]

,date,name,title,content,link


In [ ]:
news_df['name'][22][12:-38]

'헤럴드경제'

In [ ]:
df1 = pd.read_excel('/content/2022.01.01 ~ 2022.01.01.xlsx')
df1

,Unnamed: 0,date,name,title,content
0,0,2022-01-01,NaN,서울 아파트 7주째 ‘팔자〉사자’… 전세도 4주째 매물이 많아,[\n서울 매매수급지수 27개월새 최저… 전국 지수도 12주 연속 하락\n\n\n\...
1,1,2022-01-01,NaN,[2022 부스트라다무스]② 이상우 “올해 집값 10% 오를 것. 서울 아파트 14...,[\n\t\t\t “2022년에도 집값은 10% 정도 오를 것이라고 전...
2,2,2022-01-01,NaN,작년 펄펄 끓었던 부동산시장…'빙하기'로 마무리,"[\n[집값 톡톡]서울 은평·도봉·강북 아파트값 하락거래량, 금융위기 이후 최저…정..."
3,3,2022-01-01,NaN,"집값 ""더 오른다"" vs ""하락, 이미 시작""[새해 부동산 어디로①]","[\n기사내용 요약전문가 10인, 작년 13.3% 올라…4분기부터 주춤상승 전망 여..."
4,4,2022-01-01,NaN,청년들이 뽑은 최대 현안 ‘자산 격차 해소’,[\n[앵커]부동산을 중심으로 한 자산 불평등 문제도 심각하죠.집값이 너무 많이 오...
5,5,2022-01-01,NaN,올해 42만 가구 쏟아진다…둔촌·송파 등 민영 분양 줄줄이,[\n[앵커]집값이 폭등한 상황에서 내 집 마련의 꿈을 이룰 수 있는 현실적인 수단...


In [ ]:
df1.loc[0, 'content']

'[\n서울 매매수급지수 27개월새 최저… 전국 지수도 12주 연속 하락\n\n\n\n서울 아파트를 사려는 사람보다 팔려는 사람이 많은 상태가 7주 연속 계속되고 있다. 금리 인상과 대출 규제뿐만 아니라 대선 전 눈치 보기 등으로 거래절벽이 한동안 이어질 것으로 전망된다. 31일 한국부동산원에 따르면 지난해 12월 넷째 주(27일 기준) 서울 아파트 매매수급지수는 93.5로 전주(93.9)보다 0.4포인트 떨어졌다. 이는 2019년 9월 셋째 주(93.0) 이후 2년 3개월 만에 가장 낮은 수치다. 이 지수가 100을 밑돌수록 공급(매물)이 수요보다 많아 매도세가 짙다는 뜻이다.  지난해 서울 아파트 매매수급지수는 4월 첫째 주(96.1)를 제외하고 100을 웃돌았다가 11월 셋째 주(99.6) 기준선 밑으로 떨어진 뒤 현재까지 7주 연속 하락세를 보이고 있다. 이런 분위기는 전국으로 범위를 넓혀도 비슷하다. 12월 넷째 주 전국 아파트 매매수급지수는 96.1로 지난주보다 0.7포인트 하락했다. 10월 둘째 주(105.4) 이후 12주 연속 하락하면서 4주 연속 100을 밑돌고 있다.  전국 17개 시도 중 아파트 매매수급지수가 가장 낮은 세종의 이번 주 매매수급지수는 전주(81.1) 대비 2.6포인트 떨어진 78.5로 조사됐다. 2015년 3월 둘째 주(76.3) 이후 6년 9개월 만에 최저치다. 아파트 전세 시장도 위축되는 분위기다. 12월 넷째 주 서울 아파트 전세수급지수는 95.7로 전주(96.9) 대비 1.2포인트 떨어지면서 4주 연속 100 아래에 머물렀다. 전국 아파트 전세수급지수(98.7) 역시 2주 연속 100을 넘지 못했다. 고준석 동국대 법무대학원 겸임교수는 “정부의 대출 규제, 금리 인상 등으로 수요자들의 자금 여력이 부족해졌다”며 “대선을 앞두고 정책 변수가 많기 때문에 매도자들과 매수자들이 선택을 미루며 거래절벽이 심해진 점 역시 시장 위축을 부추기고 있다”고 설명했다.\n]'

In [ ]:
s_date

'2022.01.01'

In [ ]:
news_contents

['[\n수요 감소에 계약 만기 임박한 물건, 수천만원 낮춰 계약전셋값 급등·대출 규제 등에 재계약 늘고 신규 수요는 억눌린 탓약보합 전환 분위기속 일각선 4년치 인상 반영해 최고가 거래 \'대조\'(서울=연합뉴스) 서미숙 기자 = 최근 서울 아파트 매매 시장에 이어 전세 시장에서도 실거래 가격이 이전 거래에 비해 하락한 단지들이 늘어나는 분위기다.     대출 규제, 계약갱신청구권 사용 등으로 재계약은 늘고 평수를 넓히거나 거주지를 옮기려는 이동 수요는 급감하면서 \'급전세\' 계약이 증가한 것이다.     이 때문에 최근 서울 아파트 매매 가격이 일부 하락한 가운데 전셋값도 약세로 돌아서는 게 아니냐는 관측이 나온다.    반면 일부 단지와 주택형에 따라서는 역대 최고가 전세 거래도 성사돼 전월세 시장 불안의 불씨는 여전히 남아 있다는 지적도 만만치 않다. \n\n\n\n서울시내 아파트[연합뉴스 자료사진]  신규 전세 계약, 종전 대비 하락 거래 늘어     9일 서울부동산정보광장의 전세 실거래가 자료를 보면 최근 들어 신규 전세 계약으로 보이는 거래 중 종전 거래가격보다 신고금액이 하락한 경우가 속속 등장하고 있다.    최근 2년 새 계약갱신청구권 등 \'임대차 2법\' 시행으로 전셋값이 급등한 데다 금융당국이 전세자금 대출을 엄격하게 규제함에 따라 신규로 전세를 얻으려는 이동 수요가 줄어들면서 전세 물건이 적체되는 경우가 많아진 것이다. 이 때문에 전세 계약 만기가 임박한 집주인 가운데 일부는 당초 내놨던 금액보다 낮춰 전세 계약을 하는 경우가 늘고 있다.     송파구 잠실동 잠실엘스 전용 84.88㎡는 지난해 10월 최고 14억원짜리 전세계약이 2건 있었으나 12월 들어서는 최고가 거래금액이 13억원으로 낮아졌다.    잠실 리센츠 전용 59.99㎡도 지난해 8월 최고 11억8천만원까지 전세계약이 이뤄졌는데 9월 이후에는 10억원이 넘는 전세계약은 한 건도 없었고, 12월 들어서는 최고가 거래가 8억1천900만원으로 내려왔다.     잠실의 한 중

In [ ]:
news_df.loc[0, 'link']

'https://n.news.naver.com/mnews/article/001/0012950872?sid=101'

In [ ]:
news_df['name'].unique()

array(['연합뉴스', '뉴시스', 'TV조선', '머니S', '매일경제', '뉴스1', '한국경제', '더팩트',
       '매경이코노미', '아시아경제', '국민일보', '머니투데이', '서울경제', '헤럴드경제', '경향신문',
       '동아일보', '조선일보', '세계일보', 'CJB청주방송', '스포츠경향', '데일리안', '중앙일보', '주간동아',
       '조선비즈', 'SBS', '이데일리', '채널A', '아이뉴스24', '시사저널', '파이낸셜뉴스', '대전일보',
       '연합뉴스TV', '여성신문', '문화일보', 'KBS', '한국경제TV', '노컷뉴스', '디지털타임스',
       '부산일보', '한국일보', 'SBS Biz', '한경비즈니스', 'MBN', '스타뉴스', '한겨레', 'YTN'],
      dtype=object)

In [ ]:
news_df['name'].value_counts()

연합뉴스       20
뉴스1        18
뉴시스        18
아시아경제      16
한국경제       15
매일경제       13
중앙일보       13
국민일보       11
조선일보       10
이데일리        9
서울경제        9
경향신문        9
동아일보        9
머니투데이       8
파이낸셜뉴스      7
세계일보        7
조선비즈        7
헤럴드경제       7
더팩트         7
매경이코노미      6
아이뉴스24      6
연합뉴스TV      5
디지털타임스      4
머니S         4
TV조선        4
문화일보        4
데일리안        4
노컷뉴스        4
KBS         3
SBS         3
채널A         3
시사저널        3
대전일보        3
CJB청주방송     3
여성신문        3
한국일보        3
SBS Biz     2
주간동아        2
한국경제TV      2
스포츠경향       2
부산일보        1
한경비즈니스      1
MBN         1
스타뉴스        1
한겨레         1
YTN         1
Name: name, dtype: int64

In [ ]:
import numpy as np

In [ ]:
np.count(news_df['name'].values)

AttributeError: ignored

In [ ]:
news_df['name'].value_counts().sum()

292

In [ ]:
news_df.loc[249, 'content'] = '[서울=뉴시스] 5일 한국감정원에 따르면, 지난 2일 기준 서울 아파트 매매가격이 0.1% 상승 했다. 전국은 지난주 대비 상승폭이 축소됐다. (그래픽=안지혜 기자)'

In [ ]:
news_df.loc[249, 'content']

'[서울=뉴시스] 5일 한국감정원에 따르면, 지난 2일 기준 서울 아파트 매매가격이 0.1% 상승 했다. 전국은 지난주 대비 상승폭이 축소됐다. (그래픽=안지혜 기자)'

In [ ]:
news_df = news_df[news_df['name'].str.strip().astype(bool)]

In [ ]:
news_df[news_df['name'].isna()]

,date,name,title,content,link


In [ ]:
news_df[news_df['name'] == '']

,date,name,title,content,link


In [ ]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1334 entries, 0 to 1334
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     1334 non-null   object
 1   name     1334 non-null   object
 2   title    1334 non-null   object
 3   content  1334 non-null   object
 4   link     1334 non-null   object
dtypes: object(5)
memory usage: 62.5+ KB


In [ ]:
news_df.reset_index(drop = True, inplace = True)
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1334 entries, 0 to 1333
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     1334 non-null   object
 1   name     1334 non-null   object
 2   title    1334 non-null   object
 3   content  1334 non-null   object
 4   link     1334 non-null   object
dtypes: object(5)
memory usage: 52.2+ KB


In [ ]:
news_df[news_df['title'].str.contains('홈즈')]

,date,name,title,content,link


In [ ]:
outputFileName = f'{s_date} ~ {e_date}.xlsx'
news_df.to_excel(outputFileName, sheet_name='sheet1')